## Reasoning with CDS Rules


We will use a logic-based reasoning engine to apply a knowledge base of rules to particular patient descriptions.

In [2]:
import pytholog as pl

### Deciding between COVID, cold, flu and allergies

Our decision support knowledge base consists of the following rules:
- fever is present in COVID and flu
- headache is present in flu, COVID and colds
- blocked nose is present in colds and allergies
- cough in COVID and flu
- shortness of breath in COVID and allergies


In [3]:
## Create a knowledge base about respiratory disease conditions
respdis = pl.KnowledgeBase("respiratory_diseases")
respdis([
    "symptom(fever,covid)",
    "symptom(fever,flu)",
    "symptom(headache,flu)", 
    "symptom(headache,covid)",
    "symptom(headache,cold)", 
    "symptom(blockednose,cold)",
    "symptom(blockednose,allergy)",
    "symptom(cough,covid)",
    "symptom(cough,flu)",
    "symptom(shortbreath,covid)",
    "symptom(shortbreath,allergy)"
])

### Patient data

We have the following patients: 
- JM has cough and fever but no shortness of breath
- LM has shortness of breath and blocked nose
- LS has fever and headache



In [4]:
respdis([
    "has_symptom(jm,cough)",
    "has_symptom(jm,fever)",
    "has_symptom(jm,shortbreath)",
    "has_symptom(lm,blockednose)",
    "has_symptom(ls,fever)",
    "has_symptom(ls,headache)"
])


### We query our knowledge base for conditions that match the symptoms

What conditions might our patients have?  


In [5]:
respdis(["has_condition(X,D) :- has_symptom(X,Y), symptom(Y,D)"])

print(respdis.query(pl.Expr("has_condition(X,D)")))


conditions = set([ (list(y.values())[0],list(y.values())[1]) for y in respdis.query(pl.Expr("has_condition(X,D)")) ])

has_condition = dict()

for (p,c) in conditions:
    if p not in has_condition:
        has_condition[p]=[]
    if c not in has_condition[p]:
        has_condition[p].append(c)
        
has_condition

[{'X': 'jm', 'D': 'covid'}, {'X': 'jm', 'D': 'flu'}, {'X': 'jm', 'D': 'covid'}, {'X': 'jm', 'D': 'flu'}, {'X': 'jm', 'D': 'covid'}, {'X': 'jm', 'D': 'allergy'}, {'X': 'lm', 'D': 'cold'}, {'X': 'lm', 'D': 'allergy'}, {'X': 'ls', 'D': 'covid'}, {'X': 'ls', 'D': 'flu'}, {'X': 'ls', 'D': 'flu'}, {'X': 'ls', 'D': 'covid'}, {'X': 'ls', 'D': 'cold'}]


{'jm': ['allergy', 'covid', 'flu'],
 'ls': ['cold', 'covid', 'flu'],
 'lm': ['allergy', 'cold']}

### Now we exclude conditions for which symptoms are not present

Previously we could see any condition that the patient might have based on any symptom that the patient might have. Now we try to force the system to give us only the best recommendations by taking absent symptoms into account as well. 

- fever is rare in colds and allergies
- headache is not present in allergies
- cough is seldom present in colds and never in allergies
- shortness of breath is not present in colds and flu

In [6]:
respdis(["not_symptom(fever,cold)",
    "not_symptom(fever,allergy)",
    "not_symptom(headache,allergy)",
    "not_symptom(blockednose,covid)",
    "not_symptom(blockednose,flu)",
    "not_symptom(cough,cold)",
    "not_symptom(cough,allergy)",
    "not_symptom(shortbreath,cold)",
    "not_symptom(shortbreath,flu)",
    "not_condition(X,D) :- has_symptom(X,Y) , not_symptom(Y,D)"])

not_conditions = set([ (list(y.values())[0],list(y.values())[1]) for y in respdis.query(pl.Expr("not_condition(X,D)")) ])

not_has_condition = dict()

for (p,c) in not_conditions:
    if p not in not_has_condition:
        not_has_condition[p]=[]
    if c not in not_has_condition[p]:
        not_has_condition[p].append(c)
        
only_conditions = dict()

for p in has_condition.keys():
    conds = has_condition[p]
    not_conds = not_has_condition[p]
    only_conditions[p]=[x for x in conds if x not in not_conds]
    

print(respdis.query(pl.Expr("has_symptom(X,Y)")) )

    
only_conditions

[{'X': 'jm', 'Y': 'cough'}, {'X': 'jm', 'Y': 'fever'}, {'X': 'jm', 'Y': 'shortbreath'}, {'X': 'lm', 'Y': 'blockednose'}, {'X': 'ls', 'Y': 'fever'}, {'X': 'ls', 'Y': 'headache'}]


{'jm': ['covid'], 'ls': ['covid', 'flu'], 'lm': ['allergy', 'cold']}